In [6]:
from datetime import datetime
import xarray as xr

from typing import Union, List
import matplotlib.pyplot as plt

from mhwpype import ReferencePeriod
from mhwpype.utils import update_names, reformat_longitude, assign_depth
from mhwpype.core import build_heat_spike_mask
from mhwpype.core import build_mhw_mask, group_from_mask, group_stats

In [8]:
filepath = 'data/sst.day.mean.1981-2024.nc'
ds = xr.open_dataset(filepath)

ds = update_names(ds)
ds['longitude'] = reformat_longitude(ds.longitude)
ds = ds.sortby(['time','latitude','longitude'])
ds = ds.sel(latitude= slice(41.5,46.5), longitude=slice(-130,-122))
ds = assign_depth(ds,0)

ds = ds.sel(depth = 0, latitude = 44.5, longitude = -125, method='nearest')

In [22]:
fit = ds.polyfit(dim = 'time', deg = 1, skipna = True)

In [29]:
m = fit.sel(degree = 1).sea_water_temperature_polyfit_coefficients
b = fit.sel(degree = 0).sea_water_temperature_polyfit_coefficients
y = m * ds.time.astype(int) + b

In [30]:
fig, ax = plt.subplots(1,1, figsize = (12,8))
ax.plot(ds.time, ds.sea_water_temperature)
ax.plot(ds.time, y)